In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn import metrics
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
import datetime

%matplotlib inline
plt.style.use('ggplot')

In [8]:
df = pd.read_pickle('../pkl_data_2/merged_data.pkl')
df.index = df.index.set_levels([df.index.levels[0], pd.to_datetime(df.index.levels[1],format='%Y%m%d'), df.index.levels[2]])
df = df.sort_index(level='ann_date')
df.head()

-30       -29       -28       -27  \
ticker ann_date   quarternum                                           
KEY    1993-10-15 3           0.875159  0.880255  0.876433  0.876433   
       1994-01-21 4           1.138627  1.114401  1.153432  1.150740   
CCK    1995-10-18 3           0.771429  0.757147  0.771429  0.771429   
F      1996-01-31 4           0.937453  0.930725  0.943932  0.946923   
NOVL   1996-02-21 0           0.965263  0.966316  0.949474  0.995789   

                                   -26       -25       -24       -23  \
ticker ann_date   quarternum                                           
KEY    1993-10-15 3           0.896815  0.910828  0.929936  0.908280   
       1994-01-21 4           1.141319  1.119785  1.126514  1.149394   
CCK    1995-10-18 3           0.771429  0.771429  0.771429  0.771429   
F      1996-01-31 4           0.960877  0.964366  0.963369  0.966110   
NOVL   1996-02-21 0           0.965263  0.972632  0.980000  0.991579   

                                   -22       -21  ...  ern_date  \
ticker ann_date   quarternum                      ...             
KEY    1993-10-15 3           0.942675  0.960510  ...  19930930   
       1994-01-21 4           1.177658  1.183042  ...  19931231   
CCK    1995-10-18 3           0.785719  0.828571  ...  19950930   
F      1996-01-31 4           0.948168  0.959382  ...  19951231   
NOVL   1996-02-21 0           0.970526  0.983158  ...  19960131   

                              actual_ern_value  median_surprise  \
ticker ann_date   quarternum                                      
KEY    1993-10-15 3                       0.97              NaN   
       1994-01-21 4                       1.01              NaN   
CCK    1995-10-18 3                       0.38         6.717514   
F      1996-01-31 4                       0.29        19.014636   
NOVL   1996-02-21 0                       0.17              NaN   

                              mean_surprise  forecast_skew  forecast_kurt  \
ticker ann_date   quarternum                                                
KEY    1993-10-15 3                     NaN            NaN            NaN   
       1994-01-21 4                     NaN            NaN            NaN   
CCK    1995-10-18 3                6.717514            0.0       0.707107   
F      1996-01-31 4               19.014636            0.0       0.000000   
NOVL   1996-02-21 0                     NaN            NaN            NaN   

                                   std  filled_std  forecast_count  \
ticker ann_date   quarternum                                         
KEY    1993-10-15 3                NaN         NaN             1.0   
       1994-01-21 4                NaN         NaN             1.0   
CCK    1995-10-18 3           0.056569    0.056569             2.0   
F      1996-01-31 4                NaN    0.008415             1.0   
NOVL   1996-02-21 0                NaN         NaN             1.0   

                              forecast_range  
ticker ann_date   quarternum                  
KEY    1993-10-15 3                     0.00  
       1994-01-21 4                     0.00  
CCK    1995-10-18 3                     0.08  
F      1996-01-31 4                     0.00  
NOVL   1996-02-21 0                     0.00  

[5 rows x 71 columns]

In [3]:
# build target variable

q_10 = df[10].quantile(0.1)
q_90 = df[10].quantile(0.9)
t = lambda x: 0 if (x>q_10 and x<q_90) else 1
df['y'] = [t(df[10][i]) for i in range(len(df[10]))]

print('target variable y has',df['y'].value_counts()[0],'occurences of 0 and',df['y'].value_counts()[1],'occurences of 1')


target variable y has 665 occurences of 0 and 168 occurences of 1


In [4]:
# pipeline

class DataFrameSelector(BaseEstimator, TransformerMixin):
    #
    # selects features in a DataFrame
    #
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[self.attribute_names]


In [5]:
# types of training datasets 

#type_1 = 
type_2 = ['forecast_count','std','forecast_range','actual_ern_value','y']


pipeline2 = Pipeline([
                ('ft_selector',DataFrameSelector(type_2)),
                ('imputer',SimpleImputer(strategy="median"))])